# Building compiled GRIDCERF technology layers

## 1. Setup environment

### 1.1 Download GRIDCERF


Download the GRIDCERF package if you have not yet done so from here: https://doi.org/10.5281/zenodo.6601789. Please extract GRIDCERF inside the data directory of this repository as the paths in this notebook are set to that expectation.

### 1.2 Import necessary Python packages

In [1]:
import os
import glob

import yaml
import rasterio
import numpy as np
import pandas as pd


## 2. Configuration


In [2]:
# get the parent directory path to where this notebook is currently stored
root_dir = os.path.dirname(os.getcwd())

# data directory in repository
data_dir = os.path.join(root_dir, "data")

# GRIDCERF data directory from downloaded archive
gridcerf_dir = os.path.join(data_dir, "gridcerf")

# validation directory
validation_dir = os.path.join(gridcerf_dir, 'validation')

# common exclusion layers
common_dir = os.path.join(gridcerf_dir, 'common')

# directory to write compiled layers to
compiled_dir = os.path.join(gridcerf_dir, 'compiled')

# directory containing technology specific layers
tech_dir = os.path.join(gridcerf_dir, 'technology_specific')

# directory containing reference data
reference_dir = os.path.join(gridcerf_dir, 'reference')

# a file of technologies and their levels with the raster name
cerf_technologies_file = os.path.join(reference_dir, "gridcerf_technologies.txt")

land_mask_file_0 = os.path.join(reference_dir, "gridcerf_landmask_0-land.tif")
land_mask_file_1 = os.path.join(reference_dir, "gridcerf_landmask.tif")


## 3. Generate compiled technology suitability rasters

### 3.1 Functions to build suitability


In [3]:
def compile_layer(target_layer, compile_list, land_mask_file_0, land_mask_file_1, output_dir):
    """Compile a suitability layer."""
    
    land_mask_0 = rasterio.open(land_mask_file_0).read(1)

    with rasterio.open(land_mask_file_1) as template:

        metadata = template.meta.copy()

        land_mask_1 = template.read(1)

        for index, i in enumerate(compile_list):

            with rasterio.open(i) as src:

                if index == 0:
                    arr = src.read(1)
                else:
                    arr += src.read(1)

            # adjust to binary suitability with nodata set to 3 for uint8 storage 
            arr = np.where(arr == 0, 0, 1)

            # apply land mask
            arr *= land_mask_1
            
            arr += land_mask_0

            # write compiled file
            output_file = os.path.join(output_dir, target_layer)
            with rasterio.open(output_file, 'w', **metadata) as dest:

                dest.write(arr.astype(np.int16), 1)
                
                                       
    return output_file


### 3.2 Process layers

In [4]:
# dictionary of technology-specific layers to include when compiling each technology
compile_dict = {
    'gridcerf_biomass_conventional_ccs_dry.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020_buff25mi.tif',
    ],
    'gridcerf_biomass_conventional_no-ccs_pond.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020_buff25mi.tif',
    ],
    'gridcerf_biomass_conventional_no-ccs_recirculating.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020_buff25mi.tif',
    ],
    'gridcerf_biomass_conventional_no-ccs_dry.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020_buff25mi.tif',
    ],
    'gridcerf_biomass_conventional_ccs_oncethrough.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020_buff25mi.tif',
    ],
    'gridcerf_biomass_conventional_ccs_recirculating.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020_buff25mi.tif',
    ],
    'gridcerf_biomass_conventional_no-ccs_oncethrough.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020_buff25mi.tif',
    ],
    'gridcerf_biomass_igcc_with-ccs_recirculating.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_biomass_igcc_with-ccs_dry.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_biomass_igcc_with-ccs_oncethrough.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_biomass_igcc_no-ccs_recirculating.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_biomass_igcc_no-ccs_dry.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_biomass_igcc_no-ccs_oncethrough.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_coal_conventional_ccs_oncethrough.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_coalmines20km_railnodes5km_navwaters5km_conus.tif',
        'gridcerf_densely_populated_ssp2_2020_buff25mi.tif',
    ],
    'gridcerf_coal_conventional_ccs_recirculating.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_coalmines20km_railnodes5km_navwaters5km_conus.tif',
        'gridcerf_densely_populated_ssp2_2020_buff25mi.tif',
    ],
    'gridcerf_coal_conventional_no-ccs_pond.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_coalmines20km_railnodes5km_navwaters5km_conus.tif',
        'gridcerf_densely_populated_ssp2_2020_buff25mi.tif',
    ],
    'gridcerf_coal_conventional_no-ccs_oncethrough.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_coalmines20km_railnodes5km_navwaters5km_conus.tif',
        'gridcerf_densely_populated_ssp2_2020_buff25mi.tif',
    ],
    'gridcerf_coal_conventional_no-ccs_recirculating.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_coalmines20km_railnodes5km_navwaters5km_conus.tif',
        'gridcerf_densely_populated_ssp2_2020_buff25mi.tif',
    ],
    'gridcerf_coal_igcc_with-ccs_oncethrough.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_coalmines20km_railnodes5km_navwaters5km_conus.tif',
        'gridcerf_densely_populated_ssp2_2020_buff25mi.tif',
    ],
    'gridcerf_coal_igcc_no-ccs_oncethrough.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_coalmines20km_railnodes5km_navwaters5km_conus.tif',
        'gridcerf_densely_populated_ssp2_2020_buff25mi.tif',
    ],
    'gridcerf_coal_conventional_ccs_dry.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_coalmines20km_railnodes5km_navwaters5km_conus.tif',
        'gridcerf_densely_populated_ssp2_2020_buff25mi.tif',
    ],
    'gridcerf_coal_conventional_no-ccs_dry.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_coalmines20km_railnodes5km_navwaters5km_conus.tif',
        'gridcerf_densely_populated_ssp2_2020_buff25mi.tif',
    ],
    'gridcerf_coal_igcc_with-ccs_recirculating.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_coalmines20km_railnodes5km_navwaters5km_conus.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_coal_igcc_with-ccs_dry.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_coalmines20km_railnodes5km_navwaters5km_conus.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_coal_igcc_no-ccs_recirculating.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_coalmines20km_railnodes5km_navwaters5km_conus.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_coal_igcc_no-ccs_dry.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_coalmines20km_railnodes5km_navwaters5km_conus.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],    
    'gridcerf_gas_cc_ccs_dry.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',       
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_gas_cc_ccs_oncethrough.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',       
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_gas_cc_ccs_recirculating.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',        
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_gas_cc_no-ccs_oncethrough.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',    
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_gas_turbine_oncethrough.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',        
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_gas_cc_no-ccs_recirculating.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_gas_cc_no-ccs_dry.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_gas_cc_no-ccs_pond.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_gas_turbine_recirculating.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_gas_turbine_dry.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_gas_turbine_pond.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_nuclear_gen3_pond.tif': [
        'gridcerf_airports_10mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_usgs_earthquake_pga_0.3_at_2pct_in_50yrs_conus.tif',
        'gridcerf_densely_populated_ssp2_2020_nuclear.tif',
    ],
    'gridcerf_nuclear_gen3_oncethrough.tif': [
        'gridcerf_airports_10mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_usgs_earthquake_pga_0.3_at_2pct_in_50yrs_conus.tif',
        'gridcerf_densely_populated_ssp2_2020_nuclear.tif',
    ],
    'gridcerf_nuclear_gen3_recirculating.tif': [
        'gridcerf_airports_10mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_usgs_earthquake_pga_0.3_at_2pct_in_50yrs_conus.tif',
        'gridcerf_densely_populated_ssp2_2020_nuclear.tif',
    ],
    'gridcerf_refinedliquids_cc_ccs_oncethrough.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_refinedliquids_cc_no-ccs_oncethrough.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_refinedliquids_ct_pond.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_refinedliquids_ct_dry.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_refinedliquids_ct_oncethrough.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_refinedliquids_ct_recirculating.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_refinedliquids_cc_ccs_recirculating.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_refinedliquids_cc_ccs_dry.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_refinedliquids_cc_no-ccs_recirculating.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_refinedliquids_cc_no-ccs_dry.tif': [
        'gridcerf_airports_3mi_buffer_conus.tif',
        'gridcerf_usdot_railnodes_navwaters_within5km.tif',
        'gridcerf_densely_populated_ssp2_2020.tif',
    ],
    'gridcerf_solar_csp_centralized_recirculating.tif': [
        "gridcerf_nrel_solar_csp_centralized_potential.tif",
        "gridcerf_srtm_slope_10pct_or_less.tif",
        "gridcerf_densely_populated_ssp2_2020.tif"
    ],
    'gridcerf_solar_csp_centralized_dry-hybrid.tif': [
        "gridcerf_nrel_solar_csp_centralized_potential.tif",
        "gridcerf_srtm_slope_10pct_or_less.tif",
        "gridcerf_densely_populated_ssp2_2020.tif"
    ],
    'gridcerf_solar_pv_centralized.tif': [
        "gridcerf_nrel_solar_pv_centralized_potential.tif",
        "gridcerf_srtm_slope_10pct_or_less.tif",
        "gridcerf_densely_populated_ssp2_2020.tif"
    ],
    'gridcerf_wind_onshore_hubheight080m.tif': [
        "gridcerf_srtm_slope_20pct_or_less.tif",
        "gridcerf_nrel_wind_development_potential_hubheight080m_cf35.tif",
        "gridcerf_densely_populated_ssp2_2020.tif" 
    ],
    'gridcerf_wind_onshore_hubheight110m.tif': [
        "gridcerf_srtm_slope_20pct_or_less.tif",
        "gridcerf_nrel_wind_development_potential_hubheight110m_cf35.tif",
        "gridcerf_densely_populated_ssp2_2020.tif"
    ],
    'gridcerf_wind_onshore_hubheight140m.tif': [
        "gridcerf_srtm_slope_20pct_or_less.tif",
        "gridcerf_nrel_wind_development_potential_hubheight140m_cf35.tif",
        "gridcerf_densely_populated_ssp2_2020.tif"
    ]
}


In [5]:
# get a list of all common layers
common_files = glob.glob(os.path.join(common_dir, "*.tif"))

n_common = len(common_files)

print(f"There are {n_common} common layers.")


There are 20 common layers.


In [6]:
# get a list of technologies to build
techs_df = pd.read_csv(cerf_technologies_file)

n_techs = techs_df.shape[0]

tech_list = techs_df["gridcerf_name"].to_list()

print(f"There are {n_techs} technologies.")


There are 56 technologies.


In [7]:
# confirm that there are not missing technologies in the list of layers to compile
for i in tech_list:
    if i not in compile_dict:
        print(i)


In [8]:
# confirm that there are not any layers in the compile dictionary that are not accounted for technologies
for i in compile_dict.keys():
    if i not in tech_list:
        print(i)


In [9]:
# pairing of surface water requirements with each technology
file_to_water_dict = {
    'gridcerf_biomass_conventional_no-ccs_dry.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan2mgd_buffer20km.tif',
     'gridcerf_biomass_conventional_with-ccs_dry.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan2mgd_buffer20km.tif',
     'gridcerf_biomass_igcc_no-ccs_dry.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan10mgd_buffer20km.tif',
     'gridcerf_biomass_igcc_with-ccs_dry.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan55mgd_buffer20km.tif',
     'gridcerf_coal_conventional_no-ccs_dry.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan25mgd_buffer20km.tif',
     'gridcerf_coal_igcc_no-ccs_dry.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan25mgd_buffer20km.tif',
     'gridcerf_coal_igcc_with-ccs_dry.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan75mgd_buffer20km.tif',
     'gridcerf_gas_cc_no-ccs_dry.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan2mgd_buffer20km.tif',
     'gridcerf_gas_cc_with-ccs_dry.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan25mgd_buffer20km.tif',
     'gridcerf_gas_ct_no-ccs_dry.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan2mgd_buffer20km.tif',
     'gridcerf_refinedliquids_cc_no-ccs_dry.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan10mgd_buffer20km.tif',
     'gridcerf_refinedliquids_cc_with-ccs_dry.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan35mgd_buffer20km.tif',
     'gridcerf_refinedliquids_ct_no-ccs_dry.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan2mgd_buffer20km.tif',
     'gridcerf_solar_csp_centralized_dry-hybrid.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan2mgd_buffer20km.tif',
     'gridcerf_biomass_conventional_no-ccs_pond.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan10mgd_buffer20km.tif',
     'gridcerf_gas_cc_no-ccs_pond.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan95mgd_buffer20km.tif',
     'gridcerf_gas_ct_no-ccs_pond.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan10mgd_buffer20km.tif',
     'gridcerf_refinedliquids_ct_no-ccs_pond.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan40mgd_buffer20km.tif',
     'gridcerf_biomass_conventional_no-ccs_recirculating.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan10mgd_buffer20km.tif',
     'gridcerf_biomass_igcc_no-ccs_recirculating.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan75mgd_buffer20km.tif',
     'gridcerf_biomass_igcc_with-ccs_recirculating.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan110mgd_buffer20km.tif',
     'gridcerf_coal_igcc_no-ccs_recirculating.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan95mgd_buffer20km.tif',
     'gridcerf_coal_igcc_with-ccs_recirculating.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan145mgd_buffer20km.tif',
     'gridcerf_gas_cc_no-ccs_recirculating.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan70mgd_buffer20km.tif',
     'gridcerf_gas_cc_with-ccs_recirculating.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan120mgd_buffer20km.tif',
     'gridcerf_gas_ct_no-ccs_recirculating.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan25mgd_buffer20km.tif',
     'gridcerf_refinedliquids_cc_no-ccs_recirculating.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan40mgd_buffer20km.tif',
     'gridcerf_refinedliquids_cc_with-ccs_recirculating.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan70mgd_buffer20km.tif',
     'gridcerf_refinedliquids_ct_no-ccs_recirculating.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan40mgd_buffer20km.tif',
     'gridcerf_solar_csp_centralized_recirculating.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan10mgd_buffer20km.tif',
     'gridcerf_biomass_conventional_no-ccs_oncethrough.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan10mgd_buffer20km.tif',
     'gridcerf_biomass_igcc_no-ccs_oncethrough.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan75mgd_buffer20km.tif',
     'gridcerf_biomass_igcc_with-ccs_oncethrough.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan110mgd_buffer20km.tif',
     'gridcerf_coal_igcc_no-ccs_oncethrough.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan95mgd_buffer20km.tif',
     'gridcerf_coal_igcc_with-ccs_oncethrough.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan145mgd_buffer20km.tif',
     'gridcerf_gas_cc_no-ccs_oncethrough.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan70mgd_buffer20km.tif',
     'gridcerf_gas_cc_with-ccs_oncethrough.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan120mgd_buffer20km.tif',
     'gridcerf_gas_ct_no-ccs_oncethrough.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan25mgd_buffer20km.tif',
     'gridcerf_refinedliquids_cc_no-ccs_oncethrough.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan40mgd_buffer20km.tif',
     'gridcerf_refinedliquids_cc_with-ccs_oncethrough.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan70mgd_buffer20km.tif',
     'gridcerf_refinedliquids_ct_no-ccs_oncethrough.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan40mgd_buffer20km.tif',
     'gridcerf_solar_csp_centralized_oncethrough.tif': 'gridcerf_nhd2plus_surfaceflow_greaterthan10mgd_buffer20km.tif'
}


In [13]:
# build final dictionary of technology to layers supporting each technology including slope and common layers
d = {}

for i in tech_list:
    
    tally = [os.path.join(tech_dir, x) for x in compile_dict[i]]
    
    # add in common layers
    tally.extend(common_files)
    
    # add cooling water requirements file if applicable
    if i in file_to_water_dict:
        tally.append(os.path.join(tech_dir, file_to_water_dict[i]))
    
    # add slope requirement
    if "solar_csp" in i:
        tally.append(os.path.join(tech_dir,'gridcerf_srtm_slope_5pct_or_less.tif'))

    elif "solar_pv" in i:
        tally.append(os.path.join(tech_dir,'gridcerf_srtm_slope_10pct_or_less.tif'))
        
    elif "wind" in i:
        tally.append(os.path.join(tech_dir,'gridcerf_srtm_slope_20pct_or_less.tif'))
    
    else:
        tally.append(os.path.join(tech_dir,'gridcerf_srtm_slope_12pct_or_less.tif'))
        
    d[i] = tally


In [27]:
# compile each technology and write to file
for i in d.keys():
    
    print(f"Processing layer:  {i}")

    compile_layer(i, 
                  d[i],
                  land_mask_file_0,
                  land_mask_file_1,
                  compiled_dir)


Processing layer:  gridcerf_biomass_conventional_ccs_dry.tif
Processing layer:  gridcerf_biomass_conventional_ccs_oncethrough.tif
Processing layer:  gridcerf_biomass_conventional_ccs_recirculating.tif
Processing layer:  gridcerf_biomass_conventional_no-ccs_pond.tif
Processing layer:  gridcerf_biomass_conventional_no-ccs_dry.tif
Processing layer:  gridcerf_biomass_conventional_no-ccs_oncethrough.tif
Processing layer:  gridcerf_biomass_conventional_no-ccs_recirculating.tif
Processing layer:  gridcerf_biomass_igcc_with-ccs_dry.tif
Processing layer:  gridcerf_biomass_igcc_with-ccs_oncethrough.tif
Processing layer:  gridcerf_biomass_igcc_with-ccs_recirculating.tif
Processing layer:  gridcerf_biomass_igcc_no-ccs_dry.tif
Processing layer:  gridcerf_biomass_igcc_no-ccs_oncethrough.tif
Processing layer:  gridcerf_biomass_igcc_no-ccs_recirculating.tif
Processing layer:  gridcerf_coal_conventional_ccs_dry.tif
Processing layer:  gridcerf_coal_conventional_ccs_oncethrough.tif
Processing layer:  gri